# Proof of Stake

Proof of Stake renueva todo el sistema de "carrera de mineros" a "grupo de validadores sin esperanzas." Proof of Stake fue creado para solventar la principal problematica de Proof of Work (Gasto energetico).

Ya no existen los mineros, ahora los nodos seran "validadores y forjadores". En este consenso, "minar un bloque" pasa a ser "forjar un bloque", significan lo mismo. 
- El forjador se encarga de verificar las transacciones y agruparlas en un bloque, firmar ese bloque con su llave privada y enviarlo a la red.
- Los Validadores se encargan de revisar que el trabajo del forjador este correcto.
- Cuando le bloque haya sido validador. El forjador anexa el bloque a la Blockchain.

Antes, los mineros provaban su validez en la red con su trabajo (de ahi viene su nombre). Ahora, prueban su validez con su "liquidez". Cada nodo pone su dinero en la red para provar que es de confianza; si el nodo llegara a querer danar la identidad de la red, su dinero (que no es una cantidad pequena) seria destruido.

Se dice que son nodos Trustless, debido a que como nadie quiere ser un nodo corrupto y perder todo su dinero, nadie confia en nadie y se revisa el trabajo de todos.

En este consenso, entre mas dinero tenga un nodo puesto como "seguro" en la red, mas probabilidad tiene de ser el siguiente forjador del bloque. 

Cada que se escoje un nuevo forjador, se escojen tambien un numero determinado de validadores. 

Nos vamos a centrar en todo este mecanismo, poniendo un ejemplo, para despues ver como se implementa en una funcion de nuestra cadena de bloques.

Vamos a ir desarrollando poco a poco, clase por clase

 ## Nodos

Un nodo que esta participando para ser elegido como validador o como forjador, tiene que tener si o si una cuenta en la que recibiria las recompensas de la red. Una cuenta puede usar la red sin la necesidad de ser un validador. Un validor es un validador por dos motivos:
1. Por que el usuario asi lo desea
2. Por que cuenta con dinero suficiente para demostrar que es de confianza.

En Proof of Stake, se necesita un "minimo" de dinero virtual para ser validor de la red.

Podemos concluir que: Si tienes una cuenta en la Blockchain, y tienes minimo N cantidad de dinero virtual, puedes ser un validador.

Para la blockchain que vamos a desarrollar, vamos a crear una nueva clase llamada Validator. Esta clase va a tener dos atributos:
- "Tokens" por cuenta.
- La direccion de la cuenta que es Validador.

In [ ]:
from tokens import Token
from account import Account

class Validator():
    def __init__(self, account):
        self.account = account
        self.tokens = []

    def set_tokens(self, total_coins):
        """Funcion que por cada moneda en el balance, se instancia un nuevo token en la cuenta."""
        for every_coin in range(0, total_coins):
            self.tokens.append(Token(self))

    def get_tokens(self):
        """Funcion que devuelve el numero de tokens en la cuenta."""
        return len(self.tokens)

Los **tokens** son objetos dentro de una blockchain que puede tener una utilidad dentro de la misma. En este caso, nos van a ayudar para resolver el algoritmo de seleccion de forjador dentro de un grupo de validadores. Vamos a crear una clase Token.

## Tokens

In [3]:
class Token():
    def __init__(self, owner):
        self.owner = owner

La clase Token va a tener una funcion muy sencilla, pero muy importante. Tener un dueno. 

## Como se ve la clase Forjador y Testigo?

### Clase Forger

### Clase Attestor

## Algoritmo de seleccion de Forjador y Testigos.

Cada Blockchain que utilice el consenso Proof of Stake tiene un algoritmo distinto; algunos dan mas beneficio a los que casi no participan, algunas penalizan por ciertas acciones u otros dan prioridad por la antiguedad del nodo, pero en todos, la que mayor probabilidad da es tener mas dinero dentro de la red como asegurador de tu trabajo, asi que en esta blockchain unicamente vamos a tomar en cuenta ese.

In [86]:
from account import Account, Validator

validadores_del_bloque = {} # validadores del bloque confirmador
total_stacked = 0 # dinero que se almacena de los validadores

# Vamos a crear 5 cuentas. Cada una de ellas se va a instanciar como un validador.
# Si bien sabemos que el dinero no sale de la nada, ponerles un balance nos ayuda para ejemplificar 
# algo mas adelante...

charles = Account(350, 'charles')
edwin = Account(500, 'edwin')
oliver = Account(200, 'oliver')
erick = Account(90, 'erick')
sonia = Account(275, 'sonia')

# Las juntamos en una variable
lista_cuentas = [charles, edwin, oliver, erick, sonia]

# Digamos que todos quieren ser validadores, solo van a pasar aquellos que tengan mas de 100 de balance en su cuenta.
# Lo primero es hacer ese filto
for cuenta in lista_cuentas: # Bucle for que recorre cada una de las cuentas
    if cuenta.balance >= 100: # Si tiene 100 o mas de balance puede ser validador
        # Instanciar un nuevo objeto Validator.
        new_validator = Validator(cuenta) 
        
        # Variable que almacena el dinero que se va a intercambiar por tokens
        account_money = int(new_validator.account.balance)
        
        # A traves del objeto Validator, podemos acceder al objeto Account, y despues al atributo balance para restarle lo que gasto en tokens.
        new_validator.account.balance -= account_money
        
        # Se utiliza al funcion set_tokens para instanciar un numero determinado de tokens en la cuenta.
        # Si el usuario tiene 200 de balance, se van a intercambiar por 200 tokens.
        new_validator.set_tokens(account_money) 

        # Se almacena en un diccionario el validador y la cuenta.
        validadores_del_bloque.update({new_validator: account_money})
        
        # Se suma al total stackeado de la red el balance que se cambio por tokens
        total_stacked += account_money

# Al final vamos a tener una lista de validadores. (Menos Erick)
validadores_del_bloque, total_stacked

({<account.Validator at 0x2cbb2d2f9d0>: 350,
  <account.Validator at 0x2cbb2e7b250>: 500,
  <account.Validator at 0x2cbb2e87130>: 200,
  <account.Validator at 0x2cbb2e8bd90>: 275},
 1325)

In [87]:
# Su balance tambien fue restado.
for keys in validadores_del_bloque.keys():
    print(f"{keys.account.nickname}, {keys.account.balance}")

charles, 0
edwin, 0
oliver, 0
sonia, 0


Una vez que tenemos la lista neta de los validadores que van a formar parte de la forja y validacion del bloque, toca hacer un sorteo.

El mecanismo con el cual se escoje es por probabilidades y es muy similar a una tombola.

In [92]:
from random import sample, choice
from forge import Forge

# Dentro de esta celda, se va a disenar un algoritmo que escoja al siguiente forger, el siguiente ejemplo fue una implementacion propia, pero cada blockchain puede variar. 
# Cada validador tiene una cantidad finita de tokens. 
# Se van a ingresar los tokens de todos en una sola lista.
# Cada token tiene un dueno, por lo tanto, se sabe de quien es.
# Cada ticket que ingresaron da una la posibilidad de ser el siguiente forjador del bloque.
# La lista se revuelve y se escoje un elemento al azara.
# El dueno del ticket ganador pasa a ser el forjador del nuevo bloque, y los no ganadores a ser los validadores.
# 1.- Lista que almacenara todos los tickets de la "rifa".
pool = []
# 2.- Bucle que recorrera a cada validador, y añadara sus tokens a la lista general.
for validator in validadores_del_bloque.keys():
    pool += validator.tokens
# 3.- Se revuelve la lista. (Como si fuera un sorteo.)
print('Acumulando los tokens de los validadores en el servidor actual...')
print(len(pool), '- tokens acumulados.')
print('Revolviendo la lista...')
pool = sample(pool, len(pool))
print('Lista revuelta!')

# Bucle que valida que los tokens se hayan incluido bien
contador = 0
for validator in validadores_del_bloque.keys():  # Por cada validador, en la lista de validadores
    for token in pool: # Por cada ticken en el pool 
        if token.owner.account.nickname == validator.account.nickname: # Si el token pertenece al usuario, se suma uno en el contador
            contador += 1
    print(validator.account.nickname, contador) # Se imprimen los tickets de cada validador 
    contador = 0
# Fin de la validacion

ticket_winner = choice(pool)
forger = Forge(ticket_winner.owner)
print(f'El forjador del nuevo bloque sera... {forger.validator.account.nickname}')

# los validadores no ganadores del sorteo pasan a ser testigos.
# se remueve el forjador, asi solo quedan los testigos
validadores_sin_el_forjador = validadores_del_bloque.copy() # Se hace una copia de la lista que contiene a los validadores de la red
validadores_sin_el_forjador.pop(forger.validator) # Se elimina al forjador de esta transaccion
validadores_sin_el_forjador # se imprime para ver que el forjador ya no esta 

Acumulando los tokens de los validadores en el servidor actual...
1325 - tokens acumulados.
Revolviendo la lista...
Lista revuelta!
charles 350
edwin 500
oliver 200
sonia 275
El forjador del nuevo bloque sera... charles


{<account.Validator at 0x2cbb2e7b250>: 500,
 <account.Validator at 0x2cbb2e87130>: 200,
 <account.Validator at 0x2cbb2e8bd90>: 275}

In [101]:
# Ya tenemos a nuestro forjador, solo nos falta crear objetos de clase Attestor.
from attestor import Attestor
testigos = [Attestor(validador) for validador in validadores_sin_el_forjador] # Bucle for que instancia dentro de una lista objetos de tipo Attestor.
testigos

In [ ]:
# En proceso de verificar las tx...
verified_tx = forger.verify_tx(self.holding_tx)
forger.create_a_block(self.chain[-1].hash, verified_tx, _block_number)
forger.sign_block()
# el forjador manda el bloque a la red
print('### Enviando el bloque a la red...')
self.last_block = forger.broadcast_block()
print('### Iniciando atestiguamiento del bloque...')
if attestors.attest(self.last_block):
self.chain.append(self.last_block)
print("### Bloque creado. ###\n")
self.holding_tx = []
for tx in self.last_block.list_of_transactions:
tx.block = _block_number
self.verify_latest_tx()
self.send_money_to_receivers()
else:
print('No se pudo incluir el bloque.')
return
# Cada testigo va a 'atestiguar' que el forjador verifico correctamente las transacciones y su bloque esta bien hecho.
# Si esta correcto, daran su 'confirmacion'. 
# Se necesita que el 75% de los testigos den su confirmacion positiva para anexar el bloque a la cadena de bloques.